# Prédiction Résultats Matchs : Étape 1
## Préparation des données

In [1]:
import pandas as pd
import numpy as np

## Chargement des données

In [2]:
matches = pd.read_csv("C:/Users/Alban/Documents/IMDS/Projet_5A/Donnees/Matches.csv", sep=";")
elo = pd.read_csv("C:/Users/Alban/Documents/IMDS/Projet_5A/Donnees/EloRatings.csv", sep=";")

print("Matches shape:", matches.shape)
print("Elo shape:", elo.shape)

print("\nColonnes Matches :", matches.columns.tolist())
print("\nColonnes Elo :", elo.columns.tolist())

Matches shape: (228377, 42)
Elo shape: (242591, 4)

Colonnes Matches : ['Division', 'MatchDate', 'MatchTime', 'HomeTeam', 'AwayTeam', 'HomeElo', 'AwayElo', 'Form3Home', 'Form5Home', 'Form3Away', 'Form5Away', 'FTHome', 'FTAway', 'FTResult', 'HTHome', 'HTAway', 'HTResult', 'HomeShots', 'AwayShots', 'HomeTarget', 'AwayTarget', 'HomeFouls', 'AwayFouls', 'HomeCorners', 'AwayCorners', 'HomeYellow', 'AwayYellow', 'HomeRed', 'AwayRed', 'OddHome', 'OddDraw', 'OddAway', 'MaxHome', 'MaxDraw', 'MaxAway', 'Over25', 'Under25', 'MaxOver25', 'MaxUnder25', 'HandiSize', 'HandiHome', 'HandiAway']

Colonnes Elo : ['date', 'club', 'country', 'elo']


C:\Users\Alban\AppData\Local\Temp\ipykernel_9960\3477841168.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  matches = pd.read_csv("C:/Users/Alban/Documents/IMDS/Projet_5A/Donnees/Matches.csv", sep=";")


## Variable cible

In [3]:
"""
H = Home, D = Draw, A = Away
"""

mapping_result = {"H": 0, "D": 1, "A": 2}
matches["FTResult_num"] = matches["FTResult"].map(mapping_result)

print("\nRépartition des résultats :")
print(matches["FTResult_num"].value_counts(normalize=True))


Répartition des résultats :
FTResult_num
0.0    0.446321
2.0    0.288430
1.0    0.265249
Name: proportion, dtype: float64


## Valeurs NaN

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

nan_ratio = matches.isna().mean().sort_values(ascending=False)
print("Pourcentage de NaN :")
print(nan_ratio)

matches["Season"] = pd.to_datetime(matches["MatchDate"]).dt.year
nan_by_year = matches.groupby("Season").apply(lambda x: x.isna().mean())
print("\nNaN :")
print(nan_by_year[["OddHome","OddDraw","OddAway","HomeShots","HomeCorners"]].head(26))

Pourcentage de NaN :
MatchTime       0.575737
HomeTarget      0.510682
AwayTarget      0.510669
HomeFouls       0.510489
AwayFouls       0.510489
HomeCorners     0.508782
AwayCorners     0.508782
HomeShots       0.507153
AwayShots       0.507140
AwayRed         0.487177
HomeYellow      0.487173
HomeRed         0.487168
AwayYellow      0.487168
AwayElo         0.387670
HomeElo         0.387373
Under25         0.359756
MaxUnder25      0.359756
Over25          0.359752
MaxOver25       0.359752
HandiAway       0.324468
HandiHome       0.324363
HandiSize       0.323255
HTResult        0.238991
HTHome          0.238991
HTAway          0.238991
MaxHome         0.121006
MaxDraw         0.121006
MaxAway         0.121006
OddAway         0.013268
OddHome         0.013268
OddDraw         0.013268
Form3Home       0.006568
Form5Home       0.006568
Form5Away       0.006568
Form3Away       0.006568
FTHome          0.000013
FTResult        0.000013
FTAway          0.000013
FTResult_num    0.000013
Away

C:\Users\Alban\AppData\Local\Temp\ipykernel_9960\3623638549.py:8: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  matches["Season"] = pd.to_datetime(matches["MatchDate"]).dt.year
C:\Users\Alban\AppData\Local\Temp\ipykernel_9960\3623638549.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nan_by_year = matches.groupby("Season").apply(lambda x: x.isna().mean())


## Choix objectif de l'année de départ

In [5]:
critical_cols = ["OddHome", "OddDraw", "OddAway", "Form5Home", "Form5Away"]

nan_by_year = matches.groupby("Season")[critical_cols].apply(lambda x: x.notna().mean())

threshold = 0.95

valid_years = nan_by_year[(nan_by_year > threshold).all(axis=1)].index

if len(valid_years) > 0:
    year_cutoff = valid_years.min()
    print(f"Année minimale conseillée (seuil {threshold*100:.0f}%): {year_cutoff}")
else:
    print("Aucune année pour ce seuil")

print("\nComplétude :")
print(nan_by_year.head(26))  

Année minimale conseillée (seuil 95%): 2002

Complétude :
         OddHome   OddDraw   OddAway  Form5Home  Form5Away
Season                                                    
2000    0.790981  0.790981  0.790981   1.000000   1.000000
2001    0.720493  0.720493  0.720493   1.000000   1.000000
2002    0.978941  0.978941  0.978941   1.000000   1.000000
2003    0.992186  0.992186  0.992186   1.000000   1.000000
2004    0.962414  0.962414  0.962414   1.000000   1.000000
2005    0.994479  0.994479  0.994479   1.000000   1.000000
2006    0.997845  0.997845  0.997845   1.000000   1.000000
2007    0.998785  0.998785  0.998785   1.000000   1.000000
2008    0.999021  0.999021  0.999021   1.000000   1.000000
2009    0.998724  0.998724  0.998724   1.000000   1.000000
2010    0.996534  0.996534  0.996534   1.000000   1.000000
2011    0.998516  0.998516  0.998516   1.000000   1.000000
2012    0.996131  0.996131  0.996131   1.000000   1.000000
2013    0.999488  0.999488  0.999488   1.000000   1.00000

#### -> 2002 ou 2005 pour le cutoff

## Features Engineering (création de features enrichies)

In [6]:
print(matches.columns.tolist())
print(elo.columns.tolist())

['Division', 'MatchDate', 'MatchTime', 'HomeTeam', 'AwayTeam', 'HomeElo', 'AwayElo', 'Form3Home', 'Form5Home', 'Form3Away', 'Form5Away', 'FTHome', 'FTAway', 'FTResult', 'HTHome', 'HTAway', 'HTResult', 'HomeShots', 'AwayShots', 'HomeTarget', 'AwayTarget', 'HomeFouls', 'AwayFouls', 'HomeCorners', 'AwayCorners', 'HomeYellow', 'AwayYellow', 'HomeRed', 'AwayRed', 'OddHome', 'OddDraw', 'OddAway', 'MaxHome', 'MaxDraw', 'MaxAway', 'Over25', 'Under25', 'MaxOver25', 'MaxUnder25', 'HandiSize', 'HandiHome', 'HandiAway', 'FTResult_num', 'Season']
['date', 'club', 'country', 'elo']


In [7]:
# Charger les données
matches = pd.read_csv("C:/Users/Alban/Documents/IMDS/Projet_5A/Donnees/Matches.csv", 
                      sep=";", parse_dates=["MatchDate"], dayfirst=True, low_memory=False)
elo = pd.read_csv("C:/Users/Alban/Documents/IMDS/Projet_5A/Donnees/EloRatings.csv", 
                  sep=";", parse_dates=["date"], dayfirst=True)

elo = elo.sort_values(["club", "date"]).reset_index(drop=True)

def compute_elo_change(elo_df, months=1):
    col_name = f"EloChange{months}M"
    elo_df[col_name] = elo_df.groupby("club")["elo"].diff(periods=months*2)
    return elo_df

for m in [1, 3, 6]:
    elo = compute_elo_change(elo, months=m)

elo_features = ["club", "date", "elo", "EloChange1M", "EloChange3M", "EloChange6M"]
elo = elo[elo_features]

matches["MatchDate"] = pd.to_datetime(matches["MatchDate"], dayfirst=True, errors="coerce")
elo["date"] = pd.to_datetime(elo["date"], dayfirst=True, errors="coerce")

def merge_asof_team(matches, elo, team_col, prefix):
    """Applique merge_asof équipe par équipe, contre le pb de tri"""
    out = []
    for team, df_team in matches.groupby(team_col):
        df_team = df_team.sort_values("MatchDate")
        df_elo = elo[elo["club"] == team].sort_values("date")
        merged = pd.merge_asof(
            df_team,
            df_elo.rename(columns={
                "club": team_col,
                "elo": f"{prefix}EloSnap",
                "EloChange1M": f"{prefix}EloChange1M",
                "EloChange3M": f"{prefix}EloChange3M",
                "EloChange6M": f"{prefix}EloChange6M"
            }),
            by=team_col,
            left_on="MatchDate",
            right_on="date",
            direction="backward",
            allow_exact_matches=True
        )
        out.append(merged)
    return pd.concat(out, ignore_index=True)

# Merge 
matches = merge_asof_team(matches, elo, "HomeTeam", "Home")
matches = merge_asof_team(matches, elo, "AwayTeam", "Away")

matches.drop(columns=["date"], inplace=True, errors="ignore")

# momentum
matches["EloChange1M_Diff"] = matches["HomeEloChange1M"] - matches["AwayEloChange1M"]
matches["EloChange3M_Diff"] = matches["HomeEloChange3M"] - matches["AwayEloChange3M"]
matches["EloChange6M_Diff"] = matches["HomeEloChange6M"] - matches["AwayEloChange6M"]

# elo
matches["EloDiff"] = matches["HomeElo"] - matches["AwayElo"] 
matches["EloTotal"] = matches["HomeElo"] + matches["AwayElo"] 
matches["EloAdvantage"] = matches["EloDiff"] / matches["EloTotal"]

matches["EloDiff2"] = matches["HomeEloSnap"] - matches["AwayEloSnap"]
matches["EloTotal2"] = matches["HomeEloSnap"] + matches["AwayEloSnap"]
matches["EloAdvantage2"] = matches["EloDiff2"] / matches["EloTotal2"]

matches["Form3Diff"] = matches["Form3Home"] - matches["Form3Away"]
matches["Form5Diff"] = matches["Form5Home"] - matches["Form5Away"]
matches["FormMomentumHome"] = matches["Form3Home"] - (matches["Form5Home"] - matches["Form3Home"])
matches["FormMomentumAway"] = matches["Form3Away"] - (matches["Form5Away"] - matches["Form3Away"])

# Probabilités (odds)
matches["PHome"] = 1 / matches["OddHome"]
matches["PDraw"] = 1 / matches["OddDraw"]
matches["PAway"] = 1 / matches["OddAway"]

# Normalisation
matches["ProbSum"] = matches["PHome"] + matches["PDraw"] + matches["PAway"]
matches["PHome_norm"] = matches["PHome"] / matches["ProbSum"]
matches["PDraw_norm"] = matches["PDraw"] / matches["ProbSum"]
matches["PAway_norm"] = matches["PAway"] / matches["ProbSum"]

matches["ShotsDifference"] = matches["HomeShots"] - matches["AwayShots"]
matches["CornersDifference"] = matches["HomeCorners"] - matches["AwayCorners"]
matches["CardsHome"] = matches["HomeYellow"] + 2*matches["HomeRed"]
matches["CardsAway"] = matches["AwayYellow"] + 2*matches["AwayRed"]
matches["CardsDiff"] = matches["CardsHome"] - matches["CardsAway"]
matches["GameDominanceIndex"] = (matches["ShotsDifference"] + matches["CornersDifference"]) / 2

print("\nComplétude des Elo après merge_asof :")
print(matches[["HomeEloSnap", "AwayEloSnap"]].notna().mean() * 100)

print("\nAperçu des features enrichies :")
print(matches[[
    "EloDiff", "EloAdvantage", "Form5Diff",
    "PHome_norm", "GameDominanceIndex",
    "EloChange1M_Diff", "EloChange3M_Diff", "EloChange6M_Diff"
]].head())


Complétude des Elo après merge_asof :
HomeEloSnap    65.835877
AwayEloSnap    65.812232
dtype: float64

Aperçu des features enrichies :
   EloDiff  EloAdvantage  Form5Diff  PHome_norm  GameDominanceIndex  \
0      NaN           NaN        4.0    0.349630                 NaN   
1      NaN           NaN        3.0    0.463155                 NaN   
2      NaN           NaN      -11.0    0.349421                 NaN   
3      NaN           NaN       -1.0    0.427620                 NaN   
4      NaN           NaN      -11.0    0.405596                 NaN   

   EloChange1M_Diff  EloChange3M_Diff  EloChange6M_Diff  
0               NaN               NaN               NaN  
1               NaN               NaN               NaN  
2               NaN               NaN               NaN  
3               NaN               NaN               NaN  
4               NaN               NaN               NaN  


### Diagnostic des NaN Elo

In [8]:
missing_home = matches[matches["HomeEloSnap"].isna()]["HomeTeam"].unique()
missing_away = matches[matches["AwayEloSnap"].isna()]["AwayTeam"].unique()

print(f"Nb clubs sans Elo (Home): {len(missing_home)}")
print(f"Nb clubs sans Elo (Away): {len(missing_away)}")

print("\nExemples clubs sans Elo (Home):", missing_home[:20])
print("Exemples clubs sans Elo (Away):", missing_away[:20])

missing_in_elo_home = [c for c in missing_home if c not in elo["club"].unique()]
missing_in_elo_away = [c for c in missing_away if c not in elo["club"].unique()]

print("\nClubs Home absents de EloRatings:", missing_in_elo_home[:20])
print("Clubs Away absents de EloRatings:", missing_in_elo_away[:20])

Nb clubs sans Elo (Home): 578
Nb clubs sans Elo (Away): 568

Exemples clubs sans Elo (Home): ['Tirol' 'St. Polten' 'Austria Vienna' 'Wolfsberger AC' 'SK Rapid'
 'A. Lustenau' 'Grazer AK' 'A. Klagenfurt' 'Ankaragucu' 'HJK' 'KTP' 'KuPS'
 'SJK' 'Ilves' 'Honka' 'HIFK' 'VPS' 'Ekenas' 'Gnistan' 'Kallithea']
Exemples clubs sans Elo (Away): ['A. Klagenfurt' 'A. Lustenau' 'AC Oulu' 'AFC Eskilstuna'
 'AFC Telford United' 'AFC Wimbledon' 'Aarau' 'Academica Clinceni'
 'Accrington' 'Airdrie' 'Airdrie Utd' 'Ajaccio GFCO' 'Akhmat Grozny'
 'Akron Togliatti' 'Albion Rvs' 'Albirex Niigata' 'Aldershot' 'Aldosivi'
 'Alfreton Town' 'All Boys']

Clubs Home absents de EloRatings: ['Tirol', 'St. Polten', 'Austria Vienna', 'Wolfsberger AC', 'SK Rapid', 'A. Lustenau', 'Grazer AK', 'A. Klagenfurt', 'Ankaragucu', 'HJK', 'KTP', 'KuPS', 'SJK', 'Ilves', 'Honka', 'HIFK', 'VPS', 'Ekenas', 'Gnistan', 'Kallithea']
Clubs Away absents de EloRatings: ['A. Klagenfurt', 'A. Lustenau', 'AC Oulu', 'AFC Eskilstuna', 'AFC Telfor

### Solution NaN Elo (fuzzy matching)

### --> vérification mapping 

# ________________________________________________________________________________________

In [9]:
matches.to_csv("C:/Users/Alban/Documents/IMDS/Projet_5A/Donnees/Matches_enrichies.csv", index=False)

In [10]:
print(matches.columns.tolist())

['Division', 'MatchDate', 'MatchTime', 'HomeTeam', 'AwayTeam', 'HomeElo', 'AwayElo', 'Form3Home', 'Form5Home', 'Form3Away', 'Form5Away', 'FTHome', 'FTAway', 'FTResult', 'HTHome', 'HTAway', 'HTResult', 'HomeShots', 'AwayShots', 'HomeTarget', 'AwayTarget', 'HomeFouls', 'AwayFouls', 'HomeCorners', 'AwayCorners', 'HomeYellow', 'AwayYellow', 'HomeRed', 'AwayRed', 'OddHome', 'OddDraw', 'OddAway', 'MaxHome', 'MaxDraw', 'MaxAway', 'Over25', 'Under25', 'MaxOver25', 'MaxUnder25', 'HandiSize', 'HandiHome', 'HandiAway', 'date_x', 'HomeEloSnap', 'HomeEloChange1M', 'HomeEloChange3M', 'HomeEloChange6M', 'date_y', 'AwayEloSnap', 'AwayEloChange1M', 'AwayEloChange3M', 'AwayEloChange6M', 'EloChange1M_Diff', 'EloChange3M_Diff', 'EloChange6M_Diff', 'EloDiff', 'EloTotal', 'EloAdvantage', 'EloDiff2', 'EloTotal2', 'EloAdvantage2', 'Form3Diff', 'Form5Diff', 'FormMomentumHome', 'FormMomentumAway', 'PHome', 'PDraw', 'PAway', 'ProbSum', 'PHome_norm', 'PDraw_norm', 'PAway_norm', 'ShotsDifference', 'CornersDifferen

In [11]:
matches["Season"] = matches["MatchDate"].dt.year
print(matches.groupby("Season").size().tail(25))

matches = matches[matches["Season"] >= 2002]  

features_disponibles = [
    'HomeEloSnap', 'AwayEloSnap', 'EloDiff2', 'EloAdvantage2',
    'EloChange1M_Diff', 'EloChange3M_Diff', 'EloChange6M_Diff',
    'Form3Home', 'Form5Home', 'Form3Away', 'Form5Away',
    'Form3Diff', 'Form5Diff', 'PHome_norm', 'PDraw_norm', 'PAway_norm',
    'ShotsDifference', 'CornersDifference', 'CardsDiff', 'GameDominanceIndex'
]
print(f"Total features utilisables : {len(features_disponibles)}")

Season
2001     6329
2002     6173
2003     3967
2004     5534
2005     6340
2006     6959
2007     6585
2008     7148
2009     7054
2010     6925
2011     7410
2012    10597
2013    11729
2014    11993
2015    12352
2016    12100
2017    12364
2018    12042
2019    12018
2020     9562
2021    13127
2022    12168
2023    12501
2024    11148
2025     1458
dtype: int64
Total features utilisables : 20
